# Custom Data Localization Analysis

This notebook demonstrates how to use the **general-purpose raw IQ data processing pipeline** to:
1. Process raw IQ samples from any directory under `raw_data/`
2. Generate custom monitoring locations
3. Run likelihood-based localization and signal strength estimation

---

## Overview

The pipeline allows you to:
- Specify which transmitter to analyze (ustar, guesthouse, mario, moran, wasatch)
- Extract power measurements from raw IQ samples
- Automatically aggregate measurements by receiver location
- Generate monitoring_locations.yaml compatible with existing workflow

---

## Step 1: Process Raw IQ Data

First, we'll use the command-line script to process raw IQ samples and generate monitoring locations.

**Note:** This step is typically run from the command line, but we'll demonstrate it here.

In [ ]:
# Example command to process raw data
# Run this in terminal:

'''
python scripts/process_raw_data_to_monitoring.py \
    --input-dir "C:/Users/serha/raw_data/stat_rot/stat/" \
    --transmitter mario \
    --num-locations 10 \
    --output-yaml "config/monitoring_locations_mario.yaml" \
    --output-data "data/processed/mario/" \
    --dedup-threshold 20.0 \
    --min-samples 10
'''

Alternatively, you can run it directly from Python:

In [ ]:
TRANSMITTERS = ['mario', 'guesthouse']
TRANSMITTER_COMMA = ",".join(TRANSMITTERS)
TRANSMITTER_UNDERSCORE = "_".join(TRANSMITTERS)
print(TRANSMITTER_COMMA)

CUSTOM_YAML=f'../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}.yaml'

In [ ]:
# Uncomment to run processing directly
#!python ../scripts/process_raw_data_to_monitoring.py --input-dir "C:/Users/serha/raw_data/stat_rot/stat/" --transmitter {TRANSMITTER_COMMA} --num-locations 10 --output-yaml ../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}.yaml --output-data ../data/processed/{TRANSMITTER_UNDERSCORE}/ --dedup-threshold 5.0

## Step 2: Load Generated Configuration

Now we'll load the generated monitoring locations and map data.

In [ ]:
# Import necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from src.utils import load_slc_map, load_monitoring_locations, get_sensor_locations_array, load_transmitter_locations
from src.localization import (
    estimate_transmit_power_map, 
    build_covariance_matrix, 
    compute_transmitter_pmf,
    estimate_received_power_map
)
from src.visualization.spatial_plots import (
    plot_transmit_power_map,
    plot_pmf_map,
    plot_signal_estimates_map
)

from src.sparse_reconstruction.propagation_matrix import compute_propagation_matrix

print("✓ Modules imported successfully")

In [ ]:
# Load configuration
with open('../config/parameters.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded")

In [ ]:
# Load SLC map
print("Loading SLC map...")
map_data = load_slc_map(
    map_folder_dir="../",
    downsample_factor=config['spatial']['downsample_factor']
)

print(f"✓ Map loaded: shape {map_data['shape']}")
print(f"  UTM Easting range: [{map_data['UTM_long'].min():.1f}, {map_data['UTM_long'].max():.1f}] m")
print(f"  UTM Northing range: [{map_data['UTM_lat'].min():.1f}, {map_data['UTM_lat'].max():.1f}] m")

In [ ]:
# Load transmitter locations to display on plots
print("Loading transmitter locations...")
all_tx_locations = load_transmitter_locations(
    config_path='../config/transmitter_locations.yaml',
    map_data=map_data
)

# Filter to only show transmitters specified in TRANSMITTERS list
tx_locations = {name: all_tx_locations[name] for name in TRANSMITTERS if name in all_tx_locations}

print(f"✓ Loaded {len(tx_locations)} transmitter location(s) for display")
print(f"  Showing only: {', '.join(TRANSMITTERS)}")
print("\nTransmitter Locations (filtered to match TRANSMITTERS list):")
for tx_name, tx_data in tx_locations.items():
    print(f"  {tx_name:12s}: ({tx_data['latitude']:.6f}, {tx_data['longitude']:.6f}) → {tx_data['coordinates']}")

In [ ]:
# Load custom monitoring locations
# Change this to match your generated YAML file
#CUSTOM_YAML = '../config/monitoring_locations_mario.yaml'
TRANSMITTER_NAME = TRANSMITTER_UNDERSCORE

print(f"Loading monitoring locations from {CUSTOM_YAML}...")
locations_config = load_monitoring_locations(
    config_path=CUSTOM_YAML,
    map_data=map_data
)

print(f"✓ Loaded {len(locations_config['data_points'])} monitoring locations")
print(f"  UTM Zone: {locations_config['utm_zone']}{'N' if locations_config['northern_hemisphere'] else 'S'}")

# Display locations
print("\nMonitoring Locations:")
for loc in locations_config['data_points']:
    print(f"  {loc['name']:12s}: ({loc['latitude']:.5f}, {loc['longitude']:.5f}) → {loc['coordinates']}")

## Step 3: Load Processed Power Measurements

Load the averaged power measurements at each monitoring location.

In [ ]:
# Load processed data
DATA_DIR = Path(f'../data/processed/{TRANSMITTER_NAME}/')

observed_powers = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_avg_powers.npy")
power_stds = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_std_powers.npy")
sample_counts = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_sample_counts.npy")

print(f"Loaded power measurements:")
print(f"  Observed powers: {observed_powers}")
print(f"  Power range: [{observed_powers.min():.2f}, {observed_powers.max():.2f}] dB")
print(f"  Total samples: {sample_counts.sum()}")

In [ ]:
# Display summary table
summary_df = pd.read_csv(DATA_DIR / f"{TRANSMITTER_NAME}_summary.csv")
print("\nPower Measurement Summary:")
print(summary_df.to_string(index=False))

In [ ]:
data_points = get_sensor_locations_array(locations_config)

print(f"Localization input:")
print(f"  {len(data_points)} monitoring locations")
print(f"  Observed powers: {observed_powers}")
print(f"  Coordinates (pixel): {data_points}")

# Compute TIREM propagation matrix
print("Computing TIREM propagation matrix...")
# Ensure you have a valid config file at this path
tirem_config_path = '../config/tirem_parameters.yaml' 

propagation_matrix = compute_propagation_matrix(
    sensor_locations=data_points,
    map_shape=map_data['shape'],
    scale=config['spatial']['proxel_size'],
    model_type='tirem',
    config_path=tirem_config_path,
    n_jobs=-1,  # Use all cores
    verbose=True
)
print(f"✓ Propagation matrix computed: {propagation_matrix.shape}")

## Step 4: Run Localization Algorithm

Apply likelihood-based localization to estimate signal strength across the region.

In [ ]:
# Step 1: Estimate transmit power map
print("Estimating transmit power at all locations...")

transmit_power_map = estimate_transmit_power_map(
    map_shape=map_data['shape'],
    sensor_locations=data_points,
    observed_powers=observed_powers,
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    n_jobs=-1,
    propagation_matrix=propagation_matrix
)

print(f"✓ Transmit power map computed")
print(f"  Range: [{transmit_power_map.min():.1f}, {transmit_power_map.max():.1f}] dB")

In [ ]:
tx_name_list = (" & ".join(TRANSMITTERS))
#  Visualize transmit power map
fig, ax = plot_transmit_power_map(
    transmit_power_map=transmit_power_map,
    data_points=data_points,
    observed_powers=observed_powers,
    UTM_lat=map_data['UTM_lat'],
    UTM_long=map_data['UTM_long'],
    band_name=f"{tx_name_list} Transmitter",
    transmitter_locations=tx_locations  # Display transmitter locations as blue X markers
)
plt.show()

In [ ]:
# Step 2: Build covariance matrix
print("Building covariance matrix...")

cov_matrix = build_covariance_matrix(
    sensor_locations=data_points,
    sigma=config['localization']['std_deviation'],
    delta_c=config['localization']['correlation_coeff'],
    scale=config['spatial']['proxel_size']
)

print(f"✓ Covariance matrix: shape {cov_matrix.shape}")
print(f"  Condition number: {np.linalg.cond(cov_matrix):.2e}")

In [ ]:
# Step 3: Compute transmitter PMF
print("Computing transmitter probability mass function...")

pmf = compute_transmitter_pmf(
    transmit_power_map=transmit_power_map,
    sensor_locations=data_points,
    observed_powers=observed_powers,
    cov_matrix=cov_matrix,
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    propagation_matrix=propagation_matrix  # <--- Add this
)

print(f"✓ PMF computed")
print(f"  PMF sum: {np.sum(pmf):.6f}")

most_likely_idx = np.unravel_index(pmf.argmax(), pmf.shape)
print(f"  Most likely transmitter location (pixel): {most_likely_idx}")

In [ ]:
# Visualize PMF
fig, ax = plot_pmf_map(
    pmf=pmf,
    data_points=data_points,
    observed_powers=observed_powers,
    UTM_lat=map_data['UTM_lat'],
    UTM_long=map_data['UTM_long'],
    band_name=f"{tx_name_list} Transmitter",
    transmitter_locations=tx_locations  # Display transmitter locations as blue X markers
)
plt.show()

In [ ]:
# Step 4: Estimate received power map
print("Estimating received power at all locations...")

# Create target grid
rows, cols = map_data['shape']
target_grid = []
for i in range(rows):
    for j in range(cols):
        target_grid.append([i, j])
target_grid = np.array(target_grid)

signal_estimates = estimate_received_power_map(
    transmit_power_map=transmit_power_map,
    pmf=pmf,
    sensor_locations=data_points,
    target_grid=target_grid,
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    probability_threshold=3e-4,
    # propagation_matrix=propagation_matrix # Optional: won't be used for full map unless shape is NxN
)

# Reshape to 2D map
signal_estimates_2d = signal_estimates.reshape(map_data['shape'])
print(f"✓ Signal estimates computed")

In [ ]:
# Visualize signal estimates
fig, ax = plot_signal_estimates_map(
    signal_estimates=signal_estimates_2d,
    data_points=data_points,
    observed_powers=observed_powers,
    UTM_lat=map_data['UTM_lat'],
    UTM_long=map_data['UTM_long'],
    band_name=f"{tx_name_list} Transmitter",
    transmitter_locations=tx_locations  # Display transmitter locations as blue X markers
)
plt.show()

## Step 5: Evaluate Results

Calculate evaluation metrics to assess localization accuracy.

In [ ]:
from src.utils import lin_to_dB

# Calculate MSE at monitoring locations
print("Evaluation Metrics:")
print("-" * 60)

mse_scores = []
for i, loc in enumerate(locations_config['data_points']):
    actual = observed_powers[i]
    predicted = lin_to_dB(signal_estimates_2d[data_points[i, 1], data_points[i, 0]])
    error = (actual - predicted) ** 2
    mse_scores.append(error)
    print(f"  {loc['name']:15s}: Actual={actual:6.2f} dB, Predicted={predicted:6.2f} dB, MSE={error:6.2f}")

mse_scores = np.array(mse_scores)
mean_mse = np.mean(mse_scores)
variance_baseline = np.var(observed_powers)

print("-" * 60)
print(f"  Mean MSE: {mean_mse:.2f} dB²")
print(f"  Baseline variance: {variance_baseline:.2f} dB²")
print(f"  Variance reduction: {(1 - mean_mse/variance_baseline)*100:.1f}%")
print("=" * 60)

## Summary

This notebook demonstrated:
1. ✓ Processing raw IQ data to generate monitoring locations
2. ✓ Loading custom monitoring locations and power measurements
3. ✓ Running likelihood-based localization algorithm
4. ✓ Visualizing transmit power, PMF, and signal strength estimates
5. ✓ Evaluating localization accuracy

You can now use this workflow to analyze signals from any transmitter using any set of receiver locations in your raw data!